In [30]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [1]:
'''Import modules'''
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, models
from collections import Counter
from skimage import io, transform

from torchsummary import summary

import matplotlib.pyplot as plt # for plotting
import numpy as np
from time import time
import collections
import pickle
import os
import gensim
from PIL import Image

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device =", device)
print("Using", torch.cuda.device_count(), "GPUs!")
parallel = True #enable nn.DataParallel for GPU
platform = "local" #colab/local
restore = True #Restore Checkpoint
phase = "Train"

Device = cpu
Using 0 GPUs!


In [3]:
VOCAB = {}
WORD2IDX = {}
IDX2WORD = {}

In [4]:
if platform == "colab":
    path_gensim = "/content/drive/My Drive/A4/wordvec/train_captions.bin"
    path_fasttext = '/content/drive/My Drive/wordvec/cc.it.300.vec.gz'
else:
    path_gensim = "../embeddings/train_captions.bin"
    path_fasttext = ''

#fasttext_model = gensim.models.KeyedVectors.load_word2vec_format(path_fasttext, binary=False, unicode_errors='replace')
#print(fasttext_model)
#gensim_model = gensim.models.Word2Vec.load(path_gensim)
#print(gensim_model)

In [5]:
idx = 0
keys_found = 0
not_found = []
vocab_dump = VOCAB.copy()
for k in VOCAB.keys():
    if k in fasttext_model.vocab:
        keys_found += 1
        vocab_dump[k] = torch.FloatTensor(fasttext_model.wv.get_vector(k))
    else:
        vocab_dump[k] = torch.randn(300)
        not_found.append(k)

print("not found", len(not_found))

#with open('/content/drive/My Drive/A4/embeddings/trained_embed.pkl', 'wb') as handle:
#  pickle.dump(vocab_dump, handle, protocol=pickle.HIGHEST_PROTOCOL)

not found 0


In [6]:
'''class Rescale(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, image):
        h, w = image.shape[:2]
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)
        img = transform.resize(image, (new_h, new_w))
        return img


class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, image):
        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        image = image.transpose((2, 0, 1))
        return image


IMAGE_RESIZE = (256, 256)
# Sequentially compose the transforms
img_transform = transforms.Compose([Rescale(IMAGE_RESIZE), ToTensor()])'''

'class Rescale(object):\n    """Rescale the image in a sample to a given size.\n\n    Args:\n        output_size (tuple or int): Desired output size. If tuple, output is\n            matched to output_size. If int, smaller of image edges is matched\n            to output_size keeping aspect ratio the same.\n    """\n\n    def __init__(self, output_size):\n        assert isinstance(output_size, (int, tuple))\n        self.output_size = output_size\n\n    def __call__(self, image):\n        h, w = image.shape[:2]\n        if isinstance(self.output_size, int):\n            if h > w:\n                new_h, new_w = self.output_size * h / w, self.output_size\n            else:\n                new_h, new_w = self.output_size, self.output_size * w / h\n        else:\n            new_h, new_w = self.output_size\n\n        new_h, new_w = int(new_h), int(new_w)\n        img = transform.resize(image, (new_h, new_w))\n        return img\n\n\nclass ToTensor(object):\n    """Convert ndarrays in sam

In [20]:
class Resize(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, image):
        img = transforms.Resize(self.output_size)(image)
        return img


class RandomHorizontalFlip(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, image):
        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        image = image.transpose((2, 0, 1))
        return image

class Normalize(object):
    """Convert ndarrays in sample to Tensors."""
    
    def __call__(self, image):
        img_norm = transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))
        image = img_norm(img_norm)
        return image

IMAGE_RESIZE = (256, 256)
# Sequentially compose the transforms
img_transform = transforms.Compose([Resize(256), RandomHorizontalFlip(), transforms.ToTensor(), Normalize()])



In [21]:
class CaptionsPreprocessing:
    """Preprocess the captions, generate vocabulary and convert words to tensor tokens
    Args:
        captions_file_path (string): captions tsv file path
    """
    def __init__(self, captions_file_path):
        self.captions_file_path = captions_file_path

        # Read raw captions
        self.raw_captions_dict = self.read_raw_captions()

        # Preprocess captions
        self.captions_dict = self.process_captions()

        # Create vocabulary
        self.start = "<start>"
        self.end = "<end>"
        self.oov = "<unk>"
        self.pad = "<pad>"
        self.vocab = self.generate_vocabulary()
        self.word2index = self.convert_word2index()        
        self.index2word = self.convert_index2word()
        

    def read_raw_captions(self):
        """
        Returns:
            Dictionary with raw captions list keyed by image ids (integers)
        """
        captions_dict = {}
        with open(self.captions_file_path, 'r', encoding='utf-8') as f:
            for img_caption_line in f.readlines():
                img_captions = img_caption_line.strip().split('\t')
                captions_dict[int(img_captions[0])] = img_captions[1:]

        return captions_dict 

    def process_captions(self):
        """
        Use this function to generate dictionary and other preprocessing on captions
        """

        raw_captions_dict = self.raw_captions_dict 
        
        # Do the preprocessing here                
        captions_dict = raw_captions_dict

        return captions_dict

 

    def generate_vocabulary(self):
        """
        Use this function to generate dictionary and other preprocessing on captions
        """
        captions_dict = self.captions_dict

        # Generate the vocabulary
        
        all_captions = ""        
        for cap_lists in captions_dict.values():
            all_captions += " ".join(cap_lists)
        all_captions = all_captions.lower().replace(","," ,").replace("."," .").split(" ")
        
        vocab = {self.pad :1, self.oov :1, self.start :1, self.end :1}
        vocab_update = Counter(all_captions) 
        vocab_update = {k:v for k,v in vocab_update.items() if v >= freq_threshold}
        vocab.update(vocab_update)        
        vocab_size = len(vocab)
        
        if phase == "Train":
            VOCAB.update(vocab)
            if platform == "colab":
                fname = '/content/drive/My Drive/A4/dict/VOCAB.pkl'
            else:
                fname = '../dict/VOCAB.pkl'
            #if not os.path.isfile(fname):
            with open(fname, 'wb') as handle:
                pickle.dump(vocab, handle, protocol=pickle.HIGHEST_PROTOCOL)
            
        print("VOCAB SIZE =", vocab_size)
        return vocab
    
    def convert_word2index(self):
        """
        word to index converter
        """
        word2index = {}
        vocab = self.vocab
        idx = 0
        for k, v in vocab.items():
            word2index[k] = idx
            idx +=1
        if phase == "Train":
            WORD2IDX.update(word2index)
            if platform == "colab":
                fname = '/content/drive/My Drive/A4/dict/WORD2IDX.pkl'
            else:
                fname = '../dict/WORD2IDX.pkl'
            #if not os.path.isfile(fname):
            with open(fname, 'wb') as handle:
                pickle.dump(word2index, handle, protocol=pickle.HIGHEST_PROTOCOL)
        return word2index
    
    def convert_index2word(self):
        """
        index to word converter
        """
        index2word = {}
        vocab = self.vocab
        idx = 0
        
        for k, v in vocab.items():
            index2word[idx] = k
            idx +=1
        if phase == "Train":
            IDX2WORD.update(index2word)
            if platform == "colab":
                fname = '/content/drive/My Drive/A4/dict/IDX2WORD.pkl'
            else:
                fname = '../dict/IDX2WORD.pkl'
            #if not os.path.isfile(fname):
            with open(fname, 'wb') as handle:
                pickle.dump(index2word, handle, protocol=pickle.HIGHEST_PROTOCOL)
        return index2word

 

    def captions_transform(self, img_caption_list):
        """
        Use this function to generate tensor tokens for the text captions
        Args:
            img_caption_list: List of captions for a particular image
        """
        if phase == "Test":
            word2index = WORD2IDX
            vocab = VOCAB
        else:
            word2index = self.word2index
            vocab = self.vocab
            
        start = self.start
        end = self.end
        oov = self.oov
        
        processed_list = list(map(lambda x: start + " "+ x + " " + end, img_caption_list))
        processed_list = list(map(lambda x: x.lower().replace(".", " .").replace(",", " ,").split(" "), processed_list))
        processed_list = list(map(lambda x: list(map(lambda y: word2index[y] if y in vocab else word2index[oov],x)),
                                  processed_list))
        return processed_list


if platform == "colab":
    CAPTIONS_FILE_PATH = '/content/drive/My Drive/A4/train_captions.tsv'
else:
    CAPTIONS_FILE_PATH = "D:/Padhai/IIT Delhi MS(R)/2019-20 Sem II/COL774 Machine Learning/Assignment/Assignment4/train_captions.tsv"
    
embedding_dim = 300
freq_threshold = 4
captions_preprocessing_obj = CaptionsPreprocessing(CAPTIONS_FILE_PATH)

VOCAB SIZE = 8900


In [389]:
if platform == "colab":
    embed_path = '/content/drive/My Drive/A4/embeddings/trained_embed.pkl'
else:
    embed_path = '../embeddings/trained_embed.pkl'
with open(embed_path , 'rb') as handle:
    vocab_dump = pickle.load(handle)

In [391]:
'''init_weights = torch.randn(len(VOCAB), 300)
idx = 0
words = WORD2IDX.keys()
for i in range(len(words)):
    init_weights[i] = vocab_dump[IDX2WORD[i]]
init_weights.shape'''

'init_weights = torch.randn(len(VOCAB), 300)\nidx = 0\nwords = WORD2IDX.keys()\nfor i in range(len(words)):\n    init_weights[i] = vocab_dump[IDX2WORD[i]]\ninit_weights.shape'

In [22]:
class ImageCaptionsDataset(Dataset):

    def __init__(self, img_dir, captions_dict, img_transform=None, captions_transform=None):
        """
        Args:
            img_dir (string): Directory with all the images.
            captions_dict: Dictionary with captions list keyed by image ids (integers)
            img_transform (callable, optional): Optional transform to be applied
                on the image sample.

            captions_transform: (callable, optional): Optional transform to be applied
                on the caption sample (list).
        """
        self.img_dir = img_dir
        self.captions_dict = captions_dict
        self.img_transform = transforms.Compose([transforms.Resize(256), 
                                                  transforms.RandomCrop(224),transforms.RandomHorizontalFlip(),
                                                  transforms.ToTensor(),transforms.Normalize((0.485, 0.456, 0.406),
                                                                                             (0.229, 0.224, 0.225))])
        self.captions_transform = captions_transform

        self.image_ids = list(captions_dict.keys())

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, 'image_{}.jpg'.format(self.image_ids[idx]))
        #image = io.imread(img_name)
        image = Image.open(img_name).convert('RGB')
        print(type(image))
        image = self.img_transform(image)
        captions = self.captions_dict[self.image_ids[idx]]
        if self.img_transform:
            image = self.img_transform(image)

        if self.captions_transform:            
            captions = self.captions_transform(captions)
            
        sample = {'image': image, 'captions': captions}

        return sample
    
    
def custom_batch(batch):
    batch_size = len(batch)
    captions = []
    
    z = list(map(lambda b: captions.extend(b['captions']),batch))    
    x = list(map(lambda b: b['image'],batch))
    print(x[0].shape)
    captions = list(map(lambda c: torch.LongTensor(c),captions))
    captions = pad_sequence(captions, batch_first=True)
    images = torch.cat(x)
    
    sample = {'image': images, 'captions': captions}    
    return sample

In [23]:
#ENCODER

class Encoder(nn.Module):
    def __init__(self, embed_dim):
        super(Encoder, self).__init__()
        resnet101 = models.resnet34(pretrained=True, progress=True)
        for param in resnet101.parameters():
            param.requires_grad = False
        print("ResNet152 Loaded Successfully..!")
        self.resnet101 = resnet101
        #self.adaptive_pool = nn.AdaptiveAvgPool2d(output_size = (1,1))
        self.fc = nn.Linear(in_features=self.resnet101.fc.in_features, out_features=embed_dim, bias = True)
        layers = list(resnet101.children())[:-1]
        self.resnet101 = nn.Sequential(*layers)
        '''for layer in list(self.resnet101.children())[2:]:
            for params in layer.parameters():
                params.requires_grad = True'''
        self.relu = nn.LeakyReLU()

    def forward(self, x):
        x = self.resnet101(x)
        #x = self.adaptive_pool(x)
        x = x.view(x.size(0), -1)
        #x = self.relu(x)
        x = self.fc(x)
        return x
        
class AttentionBlock(nn.Module):
    def __init__(self, embed_dim, units, vocab_size):
        super(AttentionBlock, self).__init__()
        self.W1 = nn.Linear(in_features = embed_dim, out_features = units)
        self.W2 = nn.Linear(in_features=units, out_features=units)
        self.V = nn.Linear(in_features=units, out_features=1)
        self.tanh = nn.Tanh()
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, img_features, hidden):
        
        hidden = hidden.unsqueeze(dim=1)
        hidden = hidden.double()
        #print("feature and hidden shape",img_features.shape, hidden.shape)
        combined_score = self.tanh(self.W1(img_features) + self.W2(hidden))
        
        attention_weights = self.softmax(self.V(combined_score))
        context_vector = attention_weights * img_features
        context_vector = torch.sum(context_vector, dim=1)
        
        return context_vector, attention_weights    



class Decoder(nn.Module):
    def __init__(self, embed_dim, lstm_hidden_size,lstm_layers=1):
        super(Decoder, self).__init__()
        self.lstm_hidden_size = lstm_hidden_size
        self.vocab_size = len(VOCAB)
        print("VOCAB SIZE = ", self.vocab_size)
        
        self.lstm = nn.LSTM(input_size = embed_dim, hidden_size = lstm_hidden_size,
                            num_layers = lstm_layers, batch_first = True)
        
        self.linear = nn.Linear(lstm_hidden_size, self.vocab_size)        
        #self.embed = nn.Embedding.from_pretrained(init_weights)
        self.embed = nn.Embedding(self.vocab_size, embed_dim)
        
        #self.attention = AttentionBlock(embed_dim, lstm_hidden_size, self.vocab_size)

        
    def forward(self, image_features, image_captions):
        
        if phase == "Train":
            #print(image)
            image_features = torch.Tensor.repeat_interleave(image_features, repeats=5 , dim=0)
        image_features = image_features.unsqueeze(1)
        
        #hidden = torch.zeros((image_features.shape[0], self.lstm_hidden_size))
        #if device == "cuda":
        #    hidden = hidden.to(torch.device("cuda:0"))
        
        #context, attention = self.attention(image_features, hidden)
        
        embedded_captions = self.embed(image_captions)
        #print("EMBED SHAPE", embedded_captions.shape)
        #print("SHAPES BEFORE CONCAT",context.unsqueeze(dim=1).shape, embedded_captions[:,:-1].shape)
        input_lstm = torch.cat((image_features, embedded_captions[:,:-1]), dim = 1)
        
        lstm_outputs, _ = self.lstm(input_lstm)        
        lstm_outputs = self.linear(lstm_outputs)
        #print("lstm_outputs.shape", lstm_outputs.shape)
        
        
        return lstm_outputs

In [24]:
class ImageCaptionsNet(nn.Module):
    def __init__(self):
        super(ImageCaptionsNet, self).__init__()        
        ##CNN ENCODER RESNET-50        
        self.Encoder = Encoder(embed_dim = embedding_dim)
        ## RNN DECODER
        self.Decoder = Decoder(embedding_dim, units, 1)    
        

    def forward(self, x):
        image_batch, captions_batch = x
        x = self.Encoder(image_batch)
        x = self.Decoder(x, captions_batch)
        return x
    
units = 512
if restore == False:
    net = ImageCaptionsNet()
    net = net.double()
    
    if parallel == True and device != "cpu":
        print("Parallel Processing enabled")
        net = nn.DataParallel(net)

    if device == "cpu":
        print("Device to CPU")
    else:
        print("Device to CUDA")
        net = net.to(torch.device("cuda:0"))



In [25]:
'''Save and Restore Checkpoints'''
def create_checkpoint(path,model, optim_obj, loss_obj,iteration, epoch):
    checkpoint = {'epoch': epoch,
                  'iteration': iteration,
                  'model_state_dict': model.module.state_dict()}

    if platform == "colab":
        directory = '/content/drive/My Drive/A4/comp_checkpoint/'
    else:
        directory = '../comp_checkpoint/'

    torch.save(checkpoint, directory + path)
    
def restore_checkpoint(path):
    new_state_dict = collections.OrderedDict()
    if platform == "colab":
        directory = '/content/drive/My Drive/A4/comp_checkpoint/'
        checkpoint = torch.load(directory + path, map_location=torch.device('cpu'))
    else:
        directory = '../comp_checkpoint/'
        checkpoint = torch.load(directory + path, map_location=torch.device('cpu'))    
    
    epoch = checkpoint['epoch']
    new_state_dict = checkpoint['model_state_dict']
    iteration = checkpoint['iteration']
    #optimizer_state_dict = checkpoint['optimizer_state_dict']
    #loss_obj = checkpoint['loss']
    print("Iterations = {}, Epoch = {}".format(iteration, epoch))
    return new_state_dict

In [26]:
pytorch_total_params = sum(p.numel() for p in net.parameters())
pytorch_trainable_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
pytorch_total_params, pytorch_trainable_params

NameError: name 'net' is not defined

### Test Model

In [28]:
phase = "Test"
if phase == "Test":
    VOCAB.clear()
    WORD2IDX.clear()
    IDX2WORD.clear()
    if platform != 'colab':
        with open('../dict/VOCAB_comp.pkl', 'rb') as handle:
            VOCAB = pickle.load(handle)
        with open('../dict/WORD2IDX_comp.pkl', 'rb') as handle:
            WORD2IDX = pickle.load(handle)
        with open('../dict/IDX2WORD_comp.pkl', 'rb') as handle:
            IDX2WORD = pickle.load(handle)
        print("Dictionary Loaded Successfully")
    else:
        with open('/content/drive/My Drive/A4/dict/VOCAB_comp.pkl', 'rb') as handle:
            VOCAB = pickle.load(handle)
        with open('/content/drive/My Drive/A4/dict/WORD2IDX_comp.pkl', 'rb') as handle:
            WORD2IDX = pickle.load(handle)
        with open('/content/drive/My Drive/A4/dict/IDX2WORD_comp.pkl', 'rb') as handle:
            IDX2WORD = pickle.load(handle)
        print("Dictionary Loaded Successfully")

        
if platform == "colab":
    IMAGE_DIR = '/content/drive/My Drive/train/'
else:
    IMAGE_DIR = 'D:/Padhai/IIT Delhi MS(R)/2019-20 Sem II/COL774 Machine Learning/Assignment/Assignment4/public_test_images/'
if restore == True:
    net = ImageCaptionsNet()
    net = net.double()
    
    state_dict = collections.OrderedDict()
    state_dict = restore_checkpoint("Full_Model_own_finaltry.pth")
    net = ImageCaptionsNet()
    net = net.double()
    net.load_state_dict(state_dict)
    print("State Dictionary Loaded Successfully.")

    # load params
    #if device != 'cpu':
     #   net = nn.DataParallel(net)
     #   net = net.to(torch.device("cuda:0"))


# Creating the Dataset
test_dataset = ImageCaptionsDataset(
    IMAGE_DIR, captions_preprocessing_obj.captions_dict, img_transform=img_transform,
    captions_transform=captions_preprocessing_obj.captions_transform, )



def caption_image(image_feature, max_words=30):
        results = []
        states = None
        x = image_feature.unsqueeze(0)
        #print(x.shape)
        with torch.no_grad():
            for i in range(max_words):
                
                hiddens, states = net.Decoder.lstm(x, states)
                #print(hiddens.shape)
                decoder_op = net.Decoder.linear(hiddens.squeeze(1))
                print(decoder_op.shape)
                predicted_word = decoder_op.argmax(1)
                prob = max(decoder_op[0].tolist())
                #print("{} - {}".format(IDX2WORD[predicted_word.item()], prob))
                x = net.Decoder.embed(predicted_word).unsqueeze(0)
                
                results.append(predicted_word.item())
                
                if predicted_word == WORD2IDX["<end>"]:
                    break
        
        caption = [IDX2WORD[i] for i in  results]
        return ' '.join(caption)


def beam_search(img_feature, max_words=15, beam_k=3):
    
    #init with start token 
    init_caption = []
    init_caption = [[[WORD2IDX["<start>"]], float(0)]]
    
    #print(img_feature.shape)
    #img_feature = img_feature.unsqueeze(dim=1)
    while len(init_caption[0][0]) < max_words:
        temp_cap = []
        for c in init_caption:  
            #print(c[0])
            cap_pad = c[0] +  [0] * int(max_words - len(c[0]))
            cap_pad = torch.LongTensor(cap_pad)
            cap_pad = cap_pad.unsqueeze(dim=0)
            lstm_op = net.Decoder(img_feature, cap_pad)        
            lstm_op = lstm_op.reshape(max_words, lstm_op.shape[2])
        
            #TOP k prob
            #print(lstm_op.shape)
            #print(torch.argmax(lstm_op[0], dim=0).tolist())
            top_pred = torch.argmax(lstm_op, dim=1)
            #top_pred = torch.argsort(top_pred)[-beam_k:]
            print(top_pred)
            for i in range(beam_k): 
                word_idx = top_pred[i]
                prob = c[1] + lstm_op[0][word_idx]
                cap = c[0][:] + [word_idx]
                
                temp_cap.append([cap, prob])
                
        init_caption = temp_cap
        init_caption = sorted(init_caption, reverse=False, key=lambda x: x[1])[-beam_k:]
    #print(type(init_caption[-1][0]))
    temp_caption = list(map(lambda x: IDX2WORD[x], init_caption[-1][0]))
    
    pred_caption = list()
    for w in temp_caption:
        if w != '<end>':
            pred_caption.append(w)
        else:
            break
    
    return pred_caption
        
def beam_pred(image, max_words = 30, beam_k=3):
    #print(image.shape)
    x = net.Encoder(image)
    #print(x.shape)
    
    #init with start token     
    init_caption = [[[WORD2IDX["<start>"]], float(0)]]
    #print("init cap",init_caption[0][0])
    for i in range(max_words):
        if len(init_caption[0][0]) > max_words:
            break
            
        temp_cap = []
        for c in init_caption:
            cap_pad = c[0] +  [0] * int(max_words - len(c[0]))            
            cap_pad = torch.LongTensor(cap_pad)
            cap_pad = cap_pad.unsqueeze(dim=0)
            print(cap_pad.shape, cap_pad)
            lstm_predicts = net.Decoder(x, cap_pad)
            lstm_predicts = lstm_predicts.reshape(max_words, lstm_predicts.shape[2])
            
            top_beams = torch.argsort(lstm_predicts, dim=1)
            top_beams = top_beams[:, -1]
            top_beams = top_beams[:beam_k]
            
            for i in range(beam_k):                
                word_idx = top_beams[i]
                prob = c[1] + float(torch.log(lstm_predicts[i][word_idx]))
                cap = c[0][:] + [word_idx.item()]
                temp_cap.append([cap, prob])
                
        init_caption = temp_cap
        init_caption = sorted(init_caption, reverse=False, key=lambda x: x[1])[-beam_k:]
        #print("init cap", init_caption)
        temp_caption = list(map(lambda x: IDX2WORD[x], init_caption[-1][0]))
    
        pred_caption = list()
        for w in temp_caption:
            if w != '<end>':
                pred_caption.append(w)
            else:
                break

        return pred_caption  

# Define your hyperparameters

NUM_WORKERS = 0 # Parallel threads for dataloading
MAX_WORDS = 15
# Creating the DataLoader for batching purposes
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=NUM_WORKERS, 
                         collate_fn=custom_batch)
import os
if device != "cpu":
    os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
t0 = time()
pred_caps = {}
for batch_idx, sample in enumerate(test_loader):
        #print("Image_idx", batch_idx)
        image_batch, captions_batch = sample['image'], sample['captions']
        #print("AFTER", image_batch)
        #print("Original", [IDX2WORD[i] for i in captions_batch)
        #print("Cap", [IDX2WORD[int(i)] for i in captions_batch[0]])
        img_features = net.Encoder(image_batch)
        #print(img_features)
        img_features = img_features.view(-1)[torch.randperm(img_features.nelement())].view(img_features.size())
        #img_features = torch.FloatTensor(np.random.randn(1,300))
        #print(img_features[0][:4].tolist(), img_features[0][-5:].tolist())
        #print(x.shape)
        #pred_cap = beam_search(img_features)
        pred_cap = caption_image(img_features, MAX_WORDS)
        
        pred_caps[batch_idx] = pred_cap
        print("Predicted",batch_idx, pred_cap)


Dictionary Loaded Successfully
ResNet152 Loaded Successfully..!
VOCAB SIZE =  8680
Iterations = 1209, Epoch = 3
ResNet152 Loaded Successfully..!
VOCAB SIZE =  8680


RuntimeError: Error(s) in loading state_dict for ImageCaptionsNet:
	Missing key(s) in state_dict: "Encoder.resnet101.0.weight", "Encoder.resnet101.1.weight", "Encoder.resnet101.1.bias", "Encoder.resnet101.1.running_mean", "Encoder.resnet101.1.running_var", "Encoder.resnet101.4.0.conv1.weight", "Encoder.resnet101.4.0.bn1.weight", "Encoder.resnet101.4.0.bn1.bias", "Encoder.resnet101.4.0.bn1.running_mean", "Encoder.resnet101.4.0.bn1.running_var", "Encoder.resnet101.4.0.conv2.weight", "Encoder.resnet101.4.0.bn2.weight", "Encoder.resnet101.4.0.bn2.bias", "Encoder.resnet101.4.0.bn2.running_mean", "Encoder.resnet101.4.0.bn2.running_var", "Encoder.resnet101.4.1.conv1.weight", "Encoder.resnet101.4.1.bn1.weight", "Encoder.resnet101.4.1.bn1.bias", "Encoder.resnet101.4.1.bn1.running_mean", "Encoder.resnet101.4.1.bn1.running_var", "Encoder.resnet101.4.1.conv2.weight", "Encoder.resnet101.4.1.bn2.weight", "Encoder.resnet101.4.1.bn2.bias", "Encoder.resnet101.4.1.bn2.running_mean", "Encoder.resnet101.4

### TRAIN LOOP

In [ ]:
if platform == "colab":
    IMAGE_DIR = '/content/drive/My Drive/train_images/'
else:
    IMAGE_DIR = 'D:/Padhai/IIT Delhi MS(R)/2019-20 Sem II/COL774 Machine Learning/Assignment/Assignment4/train_images/'

if restore == True:
    net = ImageCaptionsNet()
    net = net.double()
    #net = net.to(torch.device("cuda:0"))
    #net = nn.DataParallel(net)
    new_state_dict = collections.OrderedDict()
    new_state_dict = restore_checkpoint("caption_chkpt_multi.pth")
    
    #net.load_state_dict(new_state_dict)
    print("State Dictionary Loaded Successfully.")
    net = nn.DataParallel(net)
    net = net.to(torch.device("cuda:0"))

# Creating the Dataset
train_dataset = ImageCaptionsDataset(
    IMAGE_DIR, captions_preprocessing_obj.captions_dict, img_transform=img_transform,
    captions_transform=captions_preprocessing_obj.captions_transform
)

# Define your hyperparameters
NUMBER_OF_EPOCHS = 3
LEARNING_RATE = 1e-2
BATCH_SIZE = 50
NUM_WORKERS = 0 # Parallel threads for dataloading
cw = torch.ones(len(VOCAB), dtype=torch.double)
cw[WORD2IDX["<pad>"]] = 0
cw = cw.to(torch.device("cuda:0"))
loss_function = nn.CrossEntropyLoss(ignore_index=WORD2IDX["<pad>"], weight=cw)
optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)
print("GLOBAL PARAMS:", NUMBER_OF_EPOCHS, BATCH_SIZE, optimizer)
loss_list = []
# Creating the DataLoader for batching purposes
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS,
                          collate_fn=custom_batch)

if device != "cpu":
    os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
    torch.backends.cudnn.benchmark = True
    #torch.cuda.set_device(1)
t0 = time()
for epoch in range(NUMBER_OF_EPOCHS):
    print("$$$$$----EPOCH {}----$$$$$$".format(epoch+1))
    iteration = 0
    '''if epoch != 0:
      LEARNING_RATE *= 0.9           
      for param_group in optimizer.param_groups:
        param_group['lr'] = LEARNING_RATE
      print("OPTIMIZER =", LEARNING_RATE, optimizer)'''

    for batch_idx, sample in enumerate(train_loader):
        iteration +=1
        
        if iteration%65 == 0:
            LEARNING_RATE *= 0.92           
            for param_group in optimizer.param_groups:
              param_group['lr'] = LEARNING_RATE
            print("OPTIMIZER =", LEARNING_RATE, optimizer)

        net.zero_grad()

        image_batch, captions_batch = sample['image'], sample['captions']
        
        #print("image_shape", image_batch.shape)
        #print("batch_shape", captions_batch.shape)
        

        # If GPU training required
        if device != "cpu":
          #print("cuda")
          image_batch, captions_batch = image_batch.to(torch.device("cuda:0")), captions_batch.to(torch.device("cuda:0"))
        
        output_captions = net((image_batch, captions_batch))
        

        #print("size for loss", output_captions.shape, captions_batch.shape)
        #torch.Size([10, 26, 9934]) torch.Size([10, 26])
        loss = loss_function(output_captions.reshape(-1, output_captions.shape[2]), captions_batch.reshape(-1))
        loss_list.append(loss.item())
        
        loss.backward()
        optimizer.step()
        
        if iteration%25 == 0:
            create_checkpoint("caption_chkpt_comp.pth", net, optimizer, loss, iteration, epoch+1)
        print("ITERATION:[{}/{}] | LOSS: {} | EPOCH = [{}/{}] | TIME ELAPSED ={}Mins".format(iteration, round(29000/BATCH_SIZE)+1,
              round(loss.item(), 6), epoch+1, NUMBER_OF_EPOCHS, round((time()-t0)/60,2)))
    print("\n$$Loss = {},EPOCH: [{}/{}]\n\n".format(round(loss.item(), 6), epoch+1, NUMBER_OF_EPOCHS))
    create_checkpoint("epoch_chkpt_comp.pth", net, optimizer, loss, iteration, epoch+1)

create_checkpoint("Final_Model_comp.pth", net, optimizer, loss, iteration, epoch+1)


In [91]:
models.resnet34()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
reference = [['this', 'is', 'a', 'test'], ['this', 'is' 'test']]
candidate = ['un', 'uomo', 'in', 'camicia', 'rossa', 'e', 'un', 'gilet', 'blu', 'e', 'una', 'donna', '.']
score = sentence_bleu(reference, candidate)

In [32]:
pred = '<start> un uomo in camicia rossa e un gilet blu e una donna , <unk> <unk> <unk> . <end>'
[VOCAB[i] for  i in pred.split(" ")]

[1, 773, 223, 374, 52, 29, 215, 773, 5, 39, 215, 440, 78, 103, 1, 1, 1, 601, 1]

In [23]:
x = torch.randn(1,10)
print(x)
print(x.view(-1)[torch.randperm(x.nelement())].view(x.size()))
print(x.view(-1)[torch.randperm(x.nelement())].view(x.size()))

print(x.view(-1)[torch.randperm(x.nelement())].view(x.size()))

tensor([[ 1.8169, -1.3020,  1.1058,  1.9061,  0.2894,  0.8495,  0.1393, -1.4722,
         -2.2089, -0.4604]])
tensor([[ 0.1393,  1.1058,  1.9061, -0.4604,  1.8169, -1.3020, -1.4722, -2.2089,
          0.8495,  0.2894]])
tensor([[ 1.9061, -1.3020,  0.1393, -0.4604,  1.8169, -2.2089,  0.2894,  0.8495,
         -1.4722,  1.1058]])
tensor([[ 0.1393,  0.8495, -2.2089, -1.3020, -1.4722,  1.9061,  1.8169,  0.2894,
         -0.4604,  1.1058]])


In [7]:
x = torch.randn(1,10)
x

tensor([[-0.7557, -0.0301,  0.5531, -0.7872, -0.1490, -0.0187, -0.6112, -1.1719,
          0.5524, -0.1333]])

In [8]:
relu = nn.ReLU(inplace=False)
y = relu(x)

In [9]:
y

tensor([[0.0000, 0.0000, 0.5531, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5524,
         0.0000]])